# Sentiment analysis using SpaCy

## 0. Text processing using SpaCy

### 0.1 Lemmatization

It turns your word to its original form.  Very common thing you wanna to do, because YouTubeVideo
do not want to confuse your model that run and running are different.

Note:  But if you use very powerful neural network like transformer, NO NEED lemmatization....

In [1]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("run ran running")

for token in doc:
    print(token.text, token.lemma_)
    
#to NOT confuse the model, you want to convert words to their lemma
#for very powerful neural network like Transformer (huggingface), NO NEED TO LEMMATIZATION, bc they understand

2023-02-02 12:49:12.490463: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


run run
ran run
running run


### 0.2 Stop words

Common preprocessing is to remove stopwords, e.g., at, in, on, etc.  Removing them helps model memorize only the keywords.

Note: In powerful network, we DON'T remove stop words

In [2]:
from spacy.lang.en.stop_words import STOP_WORDS

stopwords = list(STOP_WORDS)
print(stopwords[:5])

['in', 'eleven', 'used', 'sometime', 'nothing']


In [3]:
#let's demonstrate how to remove stopword
doc = nlp("Chaky is going to eat at Thammasat with his best friend Peter.")

In [4]:
clean_tokens = []

for token in doc:
    if token.text not in stopwords:
        clean_tokens.append(token.text)
        
clean_tokens

['Chaky', 'going', 'eat', 'Thammasat', 'best', 'friend', 'Peter', '.']

In [5]:
doc = nlp("The movie should have been good.")

clean_tokens = []

for token in doc:
    if token.text not in stopwords:
        clean_tokens.append(token.text)
        
clean_tokens  #not good

['The', 'movie', 'good', '.']

### 0.3 Removing punct

In [6]:
#removing punctuation
doc = nlp("Chaky, the teacher $  /   @ # at AIT,!!!???? likes to eat naan.")

In [7]:
# # #leverage pos tag
# for token in doc:
#     print(token.text, token.pos_)

In [8]:
token_no_punct = []

for token in doc:
    if token.pos_ != 'PUNCT' and token.pos_ != 'SPACE' and token.pos_ != 'SYM':
        token_no_punct.append(token.text)

In [9]:
token_no_punct

['Chaky',
 'the',
 'teacher',
 '@',
 '#',
 'at',
 'AIT',
 'likes',
 'to',
 'eat',
 'naan']

### 0.4 Lowercasing and unnecessary spaces

In [10]:
stripped_lowercase_tokens = []

for token in doc:
    stripped_lowercase_tokens.append(token.text.lower().strip())
    
stripped_lowercase_tokens

['chaky',
 ',',
 'the',
 'teacher',
 '$',
 '',
 '/',
 '',
 '@',
 '#',
 'at',
 'ait',
 ',',
 '!',
 '!',
 '!',
 '?',
 '?',
 '?',
 '?',
 'likes',
 'to',
 'eat',
 'naan',
 '.']

### 0.5 Combine everything

In [11]:
#nowadays, we don't preprocess anymore, especially for big models, because you lose a lot of information
#if there is something you can clean, is extra spaces or like duplicate symbols.....

#if you use ML, e.g., SVM, KNN, RF, you need to preprocess
def preprocessing(sentence):
    
    stopwords = list(STOP_WORDS)
    doc = nlp(sentence)
    cleaned_tokens = []
    
    for token in doc:
        if token.text not in stopwords and token.pos_ != 'PUNCT' and token.pos_ != 'SPACE' and \
            token.pos_ != 'SYM':
                cleaned_tokens.append(token.text)
                
    return cleaned_tokens

## 1. Let's do sentiment analysis with the help sklearn and spacy!!!

In [12]:
#import stuff
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

### 1.1 Load data

In [13]:
data_yelp   = pd.read_csv('../data/yelp_labelled.txt', sep='\t', header = None, names = ['Review', 'Sentiment'])
data_amazon = pd.read_csv('../data/amazon_labelled.txt', sep='\t', header = None, names = ['Review', 'Sentiment'])
data_imdb = pd.read_csv('../data/imdb_labelled.txt', sep='\t', header = None, names = ['Review', 'Sentiment'])

In [14]:
data_yelp.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [15]:
data_yelp.shape, data_amazon.shape, data_imdb.shape

((1000, 2), (1000, 2), (748, 2))

### 1.2 EDA

Check the mean and std; check any null values

In [16]:
data = pd.concat([data_yelp, data_amazon, data_imdb], ignore_index=True)
data.shape

(2748, 2)

In [17]:
data['Sentiment'].value_counts()

1    1386
0    1362
Name: Sentiment, dtype: int64

In [18]:
data.isna().sum()

Review       0
Sentiment    0
dtype: int64

### Countvectorizer

In [19]:
#count the frequency of words in postive and negative samples
#CountVectorizer

from sklearn.feature_extraction.text import CountVectorizer

countvec = CountVectorizer(tokenizer = preprocessing)

#let's try
corpus = [
    'Chaky is coding python     ',
    'Deep learning is very deep',
    'Are you sure about this?????',
    'please hashtag #ilovepython'
]
result   = countvec.fit_transform(corpus)

#list of tokens
print(countvec.get_feature_names_out())

#count
#rows are sentences
#columns are
print(result.toarray())

['chaky' 'coding' 'deep' 'hashtag' 'ilovepython' 'learning' 'python'
 'sure']
[[1 1 0 0 0 0 1 0]
 [0 0 2 0 0 1 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 1 1 0 0 0]]


/Users/chaklam/DSAI/dsai/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [20]:
#let's look at top words categorized by postive and negative
import numpy as np

neg_cond = data.Sentiment == 0
pos_cond = data.Sentiment == 1

neg_df   = data[neg_cond]
pos_df   = data[pos_cond]

In [21]:
#count
neg_result = countvec.fit_transform(neg_df.Review)
neg_vocabs = countvec.get_feature_names_out()

pos_result = countvec.fit_transform(pos_df.Review)
pos_vocabs = countvec.get_feature_names_out()

In [22]:
neg_result.shape

(1362, 3156)

In [23]:
#sum the counts
neg_counts = np.sum(neg_result, axis=0)
pos_counts = np.sum(pos_result, axis=0)

In [24]:
#data frame
df = pd.DataFrame(neg_counts, columns = neg_vocabs).T.sort_values(by=0, ascending=False)

In [25]:
df.head(10)

,0
1,103
bad,96
movie,95
0,92
phone,78
film,72
like,67
food,66
time,62
good,57


In [26]:
df = pd.DataFrame(pos_counts, columns = pos_vocabs).T.sort_values(by=0, ascending=False)
df.head(10)

,0
great,192
good,171
film,91
phone,87
movie,87
food,60
best,59
like,58
place,58
service,55


### TfidfVectorizer

- usually, in NLP, we don't use countvectorizer
- because it makes very frequent words a prominent feature, which we don't want to
- we want something like normalized(countvectorizer) ==> tfidvectorizer

In [28]:
tfidvec = TfidfVectorizer(tokenizer=preprocessing)

#count
neg_result   = tfidvec.fit_transform(neg_df.Review)
neg_vocabs   = tfidvec.get_feature_names_out()
pos_result   = tfidvec.fit_transform(pos_df.Review)
pos_vocabs   = tfidvec.get_feature_names_out()

#sum words across all documents
neg_counts = np.sum(neg_result, axis=0)
pos_counts = np.sum(pos_result, axis=0)

print(neg_counts.shape, pos_counts.shape)
print(neg_vocabs.shape, pos_vocabs.shape)

/Users/chaklam/DSAI/dsai/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(1, 3156) (1, 3113)
(3156,) (3113,)


In [29]:
#top ten negative terms
df = pd.DataFrame(neg_counts, columns = neg_vocabs).T.sort_values(by=0, ascending=False)
df.head(10)

,0
bad,27.328393
phone,21.732597
service,21.368633
food,20.676290
movie,18.446667
time,18.194302
place,17.815239
good,16.930500
like,16.756997
waste,15.164411


In [30]:
#top ten positive terms
df = pd.DataFrame(pos_counts, columns = pos_vocabs).T.sort_values(by=0, ascending=False)
df.head(10)

,0
great,56.711255
good,47.769436
phone,30.157413
food,22.290479
place,22.060052
service,21.794690
works,21.240647
film,20.164936
movie,19.950756
excellent,19.030793


## 2. Modeling and training

Use sklearn 

In [31]:
from sklearn.svm import LinearSVC  #here i am using machine learning, NOT deep learning

#define model
classifier = LinearSVC()
tfidvec    = TfidfVectorizer()

#define X and y
X = data['Review']
y = data['Sentiment']

#split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
print(X_train.shape)

(1923,)


In [38]:
#make pipeline
clf = Pipeline([('tfidf', tfidvec),  ('clf', classifier)])

In [39]:
# #pipeline is the same as:

# X_train_transformed = tfidvec.fit_transform(X_train)
# X_train_transformed.shape  #(words, features)
# classifier.fit(X_train_transformed, y_train)

In [40]:
#train
clf.fit(X_train, y_train)

#predict
yhat = clf.predict(X_test)

#metrics
print(classification_report(yhat, y_test))

              precision    recall  f1-score   support

           0       0.82      0.84      0.83       426
           1       0.83      0.81      0.82       399

    accuracy                           0.82       825
   macro avg       0.82      0.82      0.82       825
weighted avg       0.82      0.82      0.82       825



In [41]:
#confusion matrix
confusion_matrix(yhat, y_test)

array([[358,  68],
       [ 77, 322]])

## 3. Real-world

In [42]:
clf.predict(['Chaky loves to eat sushi.'])

array([0])

In [43]:
clf.predict(['This movie is good.'])

array([1])

In [44]:
clf.predict(['This movie should have been good.'])
#double negative is a very good test !!!!
#remember the sentiment tree bank

array([1])

In [45]:
clf.predict(['This movie is crazily amazing.'])

array([1])

In [46]:
clf.predict(['This bad movie is good.'])

array([0])